In [1]:
import tweepy
import pandas as pd
import numpy as np
import json
import sys
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.tokenize import TweetTokenizer 
import nltk

In [2]:
analyzer = SentimentIntensityAnalyzer()
tw = TweetTokenizer()

In [3]:
# Authenticate to Twitter
auth = tweepy.OAuthHandler("RgcWuY2paaASZ049J0gvqQb3E", "NEF6d3hvvClSd1iYCpIe821g0YaU76FaEkRfd3utxH70wYljQN")
auth.set_access_token("1172035635215032321-GKbT2XDpYaZLHUQWG335FHVdr1MTG2", "AGWgX1vmBgRkF7YlLpsC6WxxHWJrGpEy8ClydDHuTvRIT")
# Create API object
api = tweepy.API(auth)

In [4]:
#Check Credentials
try:
   api.verify_credentials()
   print("Authentication OK")
except:
   print("Error during authentication")

Authentication OK


## World Series Tweets

In [5]:
twitter_call = api.search(q="WorldSeries", lang="en", rpp=1000, count=500)

In [6]:
tweets = []
for tweet in twitter_call:
    row = {}
    clean = nltk.tokenize.casual.remove_handles(tweet.text)
    row['Tweet'] = clean
    row ['Time'] = tweet.created_at
    row['User Name'] = tweet.user.name
    row['Score'] = analyzer.polarity_scores(clean)
    row['Location'] = tweet.user.location
    tweets.append(row)
    
print(tweets)

[{'Tweet': '  Dear  ,\nWin or lose, you will always be my first sports job love!😍\n#WinTheFight… https://t.co/F8Rww74siD', 'Time': datetime.datetime(2019, 10, 31, 0, 11, 49), 'User Name': 'FOP30', 'Score': {'neg': 0.126, 'neu': 0.561, 'pos': 0.313, 'compound': 0.6114}, 'Location': 'Wakanda'}, {'Tweet': 'RT  : Hey #Expos fans .....!!!! Who are you pulling for ? #WorldSeries #Astros #Nationals #MLB', 'Time': datetime.datetime(2019, 10, 31, 0, 11, 49), 'User Name': 'Chris Edge Sports', 'Score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, 'Location': 'California'}, {'Tweet': 'Why is Zack Greinke’s hat always too big for his head? #WorldSeries', 'Time': datetime.datetime(2019, 10, 31, 0, 11, 49), 'User Name': 'Aline VanDeventer', 'Score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, 'Location': ''}, {'Tweet': 'If the Karate Kid is watching, so am I!!! #WorldSeries #GameSeven #GoExpos https://t.co/YWl8zRercB', 'Time': datetime.datetime(2019, 10, 31, 0, 11, 48), 'User Nam

In [7]:
world_series_df = pd.DataFrame(tweets)

In [8]:
world_series_list = []
for score in world_series_df['Score'].iteritems():
    world_series_scores = (score[1].get('compound', 0)) 
    world_series_list.append(world_series_scores)

In [9]:
world_series_df['Compound Score'] = world_series_list
world_series_table = world_series_df.drop(columns=['User Name'])
world_series_table

,Tweet,Time,Score,Location,Compound Score
0,"Dear ,\nWin or lose, you will always be my ...",2019-10-31 00:11:49,"{'neg': 0.126, 'neu': 0.561, 'pos': 0.313, 'co...",Wakanda,0.6114
1,RT : Hey #Expos fans .....!!!! Who are you pu...,2019-10-31 00:11:49,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",California,0.0000
2,Why is Zack Greinke’s hat always too big for h...,2019-10-31 00:11:49,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",,0.0000
3,"If the Karate Kid is watching, so am I!!! #Wor...",2019-10-31 00:11:48,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",Canada,0.0000
4,Game 7.\n#WorldSeries,2019-10-31 00:11:48,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","Tacoma, WA",0.0000
5,Say goodbye to your fingernails.\n\n#Astros #T...,2019-10-31 00:11:48,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","Houston, TX",0.0000
6,Going to need a second source on this #WorldSe...,2019-10-31 00:11:48,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","Forest, VA",0.0000
7,LET'S GO STROS #TakeItBack #WorldSeries #Game7,2019-10-31 00:11:48,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","Spring, TX",0.0000
8,RT : SEASON FINALE. LET'S GO. \n\n#WorldSerie...,2019-10-31 00:11:47,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",HIROI SEKAI,0.0000
9,Play ball! #WorldSeries,2019-10-31 00:11:47,"{'neg': 0.0, 'neu': 0.426, 'pos': 0.574, 'comp...","Raleigh, NC",0.4003


In [10]:
world_series_table.to_csv('Data/world_series_df.csv')

In [11]:
# Scores categories
positive_WorldSeries = []
negative_WorldSeries = []
neutral_WorldSeries = []

In [12]:
for c_score in world_series_table['Compound Score']:
    if c_score > 0:
        positive_WorldSeries.append(c_score)
    elif c_score < 0:
        negative_WorldSeries.append(c_score)
    else: neutral_WorldSeries.append(c_score)

In [13]:
total_positive_WorldSeries = len(positive_WorldSeries)
total_negative_WorldSeries = len(negative_WorldSeries)
avg_positive_WorldSeries = sum(positive_WorldSeries) / float(len(positive_WorldSeries))
avg_negative_WorldSeries = sum(negative_WorldSeries) / float(len(negative_WorldSeries))
avg_neutral_WorldSeries = float(len(neutral_WorldSeries))

print(f'The total number of positive tweets for the World Series is {total_positive_WorldSeries} and the average positive tweets score is {avg_positive_WorldSeries}')
print(f'The total number of negative tweets for the World Series is {total_negative_WorldSeries} and the average negative tweets score is {avg_negative_WorldSeries}')
print(f'The total number of neutral tweets for the World Series is {avg_neutral_WorldSeries}')
  

The total number of positive tweets for the World Series is 39 and the average positive tweets score is 0.5861769230769229
The total number of negative tweets for the World Series is 5 and the average negative tweets score is -0.4651199999999999
The total number of neutral tweets for the World Series is 56.0


## Astros Tweets

In [14]:
twitter_call = api.search(q="Astros", lang="en", rpp=1000, count=500)

In [15]:
tweets = []
for tweet in twitter_call:
    row = {}
    clean = nltk.tokenize.casual.remove_handles(tweet.text)
    row['Tweet'] = clean
    row ['Time'] = tweet.created_at
    row['User Name'] = tweet.user.name
    row['Score'] = analyzer.polarity_scores(clean)
    row['Location'] = tweet.user.location
    tweets.append(row)
    
print(tweets)

[{'Tweet': 'LETS GET READY TO RUUUUUUUMMMBLLLLLLEEEE!! Astros Nats cest partie!! https://t.co/EIMPnv3Pdg', 'Time': datetime.datetime(2019, 10, 31, 0, 11, 57), 'User Name': '@KMT direction les world series (Juan Soto love u)', 'Score': {'neg': 0.0, 'neu': 0.672, 'pos': 0.328, 'compound': 0.6598}, 'Location': ''}, {'Tweet': 'in bed with nothing but my Astros hat on', 'Time': datetime.datetime(2019, 10, 31, 0, 11, 57), 'User Name': 'Chad 🐲', 'Score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, 'Location': 'Houston, TX'}, {'Tweet': 'Going with Hallie’s triple olive 🍸 tonight since the series is in the line #Astros #TakeItBack https://t.co/a60ZpVAfSN', 'Time': datetime.datetime(2019, 10, 31, 0, 11, 57), 'User Name': 'Luis Garza', 'Score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, 'Location': ''}, {'Tweet': 'RT  : DC Romeo Crennel ends his press conference with ‘Go Astros.’ The Astros are 4-0 this postseason when Crennel ends his pre…', 'Time': datetime.datetime(2019,

In [16]:
astros_df = pd.DataFrame(tweets)

In [17]:
# Get compound scores from dictionary
astros_list = []
for score in astros_df['Score'].iteritems():
    astros_scores = (score[1].get('compound', 0)) 
    astros_list.append(astros_scores)

In [18]:
# Create new column with compound scores
astros_df['Compound Score'] = astros_list
astros_table = astros_df.drop(columns=['User Name'])
astros_table

,Tweet,Time,Score,Location,Compound Score
0,LETS GET READY TO RUUUUUUUMMMBLLLLLLEEEE!! Ast...,2019-10-31 00:11:57,"{'neg': 0.0, 'neu': 0.672, 'pos': 0.328, 'comp...",,0.6598
1,in bed with nothing but my Astros hat on,2019-10-31 00:11:57,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","Houston, TX",0.0000
2,Going with Hallie’s triple olive 🍸 tonight sin...,2019-10-31 00:11:57,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",,0.0000
3,RT : DC Romeo Crennel ends his press conferen...,2019-10-31 00:11:57,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",,0.0000
4,RT : no offense but the Astros winning the Wo...,2019-10-31 00:11:57,"{'neg': 0.262, 'neu': 0.565, 'pos': 0.173, 'co...",London/Houston,-0.0900
5,RT : Astros stadium has a Torchys and a Pluck...,2019-10-31 00:11:57,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","DDD, Texas",0.0000
6,RT : One more time. \n\n#TakeItBack https://t...,2019-10-31 00:11:56,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",956,0.0000
7,Championship Orange 🍊 #Astros,2019-10-31 00:11:56,"{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'comp...","Texas, USA",0.4404
8,WINNNNNNER TAKE ALLLL\n\nI don’t think y’all H...,2019-10-31 00:11:56,"{'neg': 0.0, 'neu': 0.823, 'pos': 0.177, 'comp...","Greater Greenspoint, Houston",0.4995
9,"RT : If the Astros win tonight, we'll follow ...",2019-10-31 00:11:55,"{'neg': 0.0, 'neu': 0.774, 'pos': 0.226, 'comp...",tx,0.5859


In [19]:
astros_table.to_csv('Data/astros_df.csv')

In [20]:
# Scores categories
positive_astros = []
negative_astros = []
neutral_astros = []

In [21]:
for c_score in astros_table['Compound Score']:
    if c_score > 0:
        positive_astros.append(c_score)
    elif c_score < 0:
        negative_astros.append(c_score)
    else: neutral_astros.append(c_score)


In [22]:
total_positive_astros = len(positive_astros)
total_negative_astros = len(negative_astros)
avg_positive_astros = sum(positive_astros) / float(len(positive_astros))
avg_negative_astros = sum(negative_astros) / float(len(negative_astros))
avg_neutral_astros = float(len(neutral_astros))

print(f'The total number of positive tweets for Astros is {total_positive_astros} and the average positive tweets score is {avg_positive_astros}')
print(f'The total number of negative tweets for Astros is {total_negative_astros} and the average negative tweets score is {avg_negative_astros}')
print(f'The total number of neutral tweets for Astros is {avg_neutral_astros}')
  

The total number of positive tweets for Astros is 45 and the average positive tweets score is 0.5215466666666667
The total number of negative tweets for Astros is 13 and the average negative tweets score is -0.31179999999999997
The total number of neutral tweets for Astros is 42.0


## Nationals Tweets

In [23]:
twitter_call = api.search(q="Nationals", lang="en", rpp=1000, count=500)

In [24]:
tweets = []
for tweet in twitter_call:
    row = {}
    clean = nltk.tokenize.casual.remove_handles(tweet.text)
    row['Tweet'] = clean
    row ['Time'] = tweet.created_at
    row['User Name'] = tweet.user.name
    row['Score'] = analyzer.polarity_scores(clean)
    row['Location'] = tweet.user.location
    tweets.append(row)
    
print(tweets)

[{'Tweet': "RT  : Go 1-0 today and we'll be 1-0 forever.\n\n#WorldSeries // #STAYINTHEFIGHT https://t.co/TEQNAcc44e", 'Time': datetime.datetime(2019, 10, 31, 0, 11, 57), 'User Name': 'Q (Redskins 1-7)', 'Score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, 'Location': 'DC | Harlem, NY 📍'}, {'Tweet': '        Hay honey😍', 'Time': datetime.datetime(2019, 10, 31, 0, 11, 57), 'User Name': 'BLAKWEEN💋', 'Score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, 'Location': '🌎'}, {'Tweet': '  Go Stros!!!!!   Fuck the Nationals after all the booooos!', 'Time': datetime.datetime(2019, 10, 31, 0, 11, 57), 'User Name': 'JWall1013', 'Score': {'neg': 0.368, 'neu': 0.632, 'pos': 0.0, 'compound': -0.6876}, 'Location': 'Wisconsin, USA'}, {'Tweet': 'RT  : GOOD LUCK TONIGHT  \n\n#FINISHTHEFIGHT https://t.co/uwkzA5qrTs', 'Time': datetime.datetime(2019, 10, 31, 0, 11, 57), 'User Name': 'DC SPORTS', 'Score': {'neg': 0.0, 'neu': 0.352, 'pos': 0.648, 'compound': 0.8109}, 'Location': ''}, {'Twe

In [25]:
nationals_df = pd.DataFrame(tweets)

In [26]:
nationals_list = []
for score in nationals_df['Score'].iteritems():
    nationals_scores = (score[1].get('compound', 0)) 
    nationals_list.append(nationals_scores)

In [27]:
nationals_df['Compound Score'] = nationals_list
nationals_table = nationals_df.drop(columns=['User Name'])
nationals_table

,Tweet,Time,Score,Location,Compound Score
0,RT : Go 1-0 today and we'll be 1-0 forever.\n...,2019-10-31 00:11:57,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","DC | Harlem, NY 📍",0.0000
1,Hay honey😍,2019-10-31 00:11:57,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",🌎,0.0000
2,Go Stros!!!!! Fuck the Nationals after all...,2019-10-31 00:11:57,"{'neg': 0.368, 'neu': 0.632, 'pos': 0.0, 'comp...","Wisconsin, USA",-0.6876
3,RT : GOOD LUCK TONIGHT \n\n#FINISHTHEFIGHT h...,2019-10-31 00:11:57,"{'neg': 0.0, 'neu': 0.352, 'pos': 0.648, 'comp...",,0.8109
4,Best night of the year!! ⚾️⚾️⚾️⚾️⚾️ https://t....,2019-10-31 00:11:56,"{'neg': 0.0, 'neu': 0.556, 'pos': 0.444, 'comp...",Southern Middle Tennessee,0.6988
5,RT : Go 1-0 today and we'll be 1-0 forever.\n...,2019-10-31 00:11:56,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",BOSSUP EMPIRE,0.0000
6,RT : This is the first Game 7 in Expos/Nation...,2019-10-31 00:11:56,"{'neg': 0.102, 'neu': 0.898, 'pos': 0.0, 'comp...",Philadelphia,-0.2584
7,How will Trump take credit for the Nation...,2019-10-31 00:11:56,"{'neg': 0.053, 'neu': 0.664, 'pos': 0.283, 'co...",,0.7351
8,And I really really reeeeaaaalllyyy wanna be 1...,2019-10-31 00:11:55,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",on some course 🏌🏾,0.0000
9,"8-7, 10 innings — Nationals. Zimmerman wins it...",2019-10-31 00:11:55,"{'neg': 0.0, 'neu': 0.778, 'pos': 0.222, 'comp...","North Chevy Chase, MD",0.5719


In [28]:
nationals_table.to_csv('Data/nationals_df.csv')

In [29]:
# Scores categories
positive_nationals = []
negative_nationals = []
neutral_nationals = []

In [30]:
for c_score in nationals_table['Compound Score']:
    if c_score > 0:
        positive_nationals.append(c_score)
    elif c_score < 0:
        negative_nationals.append(c_score)
    else: neutral_nationals.append(c_score)

In [31]:
total_positive_nationals = len(positive_nationals)
total_negative_nationals = len(negative_nationals)
avg_positive_nationals = sum(positive_nationals) / float(len(positive_nationals))
avg_negative_nationals = sum(negative_nationals) / float(len(negative_nationals))
avg_neutral_nationals = float(len(neutral_nationals))

print(f'The total number of positive tweets for Nationals is {total_positive_nationals} and the average positive tweets score is {avg_positive_nationals}')
print(f'The total number of negative tweets for Nationals is {total_negative_nationals} and the average negative tweets score is {avg_negative_nationals}')
print(f'The total number of neutral tweets for Nationals is {avg_neutral_nationals}')
  

The total number of positive tweets for Nationals is 42 and the average positive tweets score is 0.5903928571428569
The total number of negative tweets for Nationals is 9 and the average negative tweets score is -0.4028444444444444
The total number of neutral tweets for Nationals is 49.0
